In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import warnings
def noop(*args, **kwargs): pass
warnings.warn = noop

In [3]:
from collections import ChainMap
from multiprocessing import cpu_count

In [41]:
from IPython.display import display
import lightgbm as lgb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.externals.joblib import Parallel, delayed
from sklearn.model_selection import train_test_split, ParameterSampler
from sklearn.ensemble import RandomForestClassifier
from tsfresh import extract_features, extract_relevant_features
from tqdm import tqdm_notebook as tqdm

In [5]:
from basedir import SAMPLE
from utils import from_feather, to_feather, kfolds 

In [6]:
seed = 1
np.random.seed(seed)

In [7]:
x_trn, y_trn, x_tst = from_feather('x_trn', 'y_trn', 'x_tst')

In [12]:
from tsfresh.feature_extraction.feature_calculators import (
    mean, median, standard_deviation, variance, skewness, kurtosis,
    mean_abs_change, mean_change, mean_second_derivative_central, 
    quantile, autocorrelation, agg_autocorrelation, partial_autocorrelation,
    abs_energy, count_above_mean, count_below_mean, maximum, minimum,
    first_location_of_minimum, first_location_of_maximum, linear_trend,
    sample_entropy, c3, 
    longest_strike_below_mean, longest_strike_above_mean, 
    number_peaks, sum_of_reoccurring_data_points, sum_values,
    large_standard_deviation,
    number_crossing_m, value_count, range_count,
    ratio_beyond_r_sigma, index_mass_quantile,
    symmetry_looking
)
from tsfresh.feature_selection.relevance import calculate_relevance_table

In [9]:
def stat(f, **params):
    def wrapper(x):
        return f(x, **params)
    wrapper.__name__ = f.__name__
    return wrapper

In [10]:
def get_series(data, ser_id, *ser_ids):
    ids = [ser_id] + list(ser_ids)
    return data[data.series_id.isin(ids)].copy()

In [11]:
default_stats = (
    mean, median, standard_deviation, variance, skewness, kurtosis, maximum, minimum,
    mean_change, mean_abs_change, count_above_mean, count_below_mean,
    mean_second_derivative_central, sum_of_reoccurring_data_points, 
    abs_energy, sum_values, sample_entropy,
    longest_strike_above_mean, longest_strike_below_mean,
    first_location_of_minimum, first_location_of_maximum,
    *[stat(large_standard_deviation, r=r*0.05) for r in range(1, 20)],
    *[stat(autocorrelation, lag=lag) for lag in range(1, 25)], 
    *[stat(number_peaks, n=n) for n in (1, 2, 3, 5, 7, 10, 25, 50)],
    *[stat(c3, lag=lag) for lag in range(1, 5)],
    *[stat(quantile, q=q) for q in (.1, .2, .3, .4, .5, .6, .7, .8, .9)],
    stat(partial_autocorrelation, param=[{'lag': lag} for lag in range(25)]),
    stat(agg_autocorrelation, param=[{'f_agg': s, 'maxlag': 40} for s in ('mean', 'median', 'var')]),
    stat(linear_trend, param=[
        {'attr': a} for a in ('pvalue', 'rvalue', 'intercept', 'slope', 'stderr')]),
    *[stat(number_crossing_m, m=m) for m in (-1, 0, 1)],
    *[stat(value_count, value=v) for v in (-1, 0, 1)],
    *[stat(range_count, min=lo, max=hi) for lo, hi in ((-1, 1), (1e12, 0), (0, 1e12))],
    *[stat(ratio_beyond_r_sigma, r=r) for r in (0.5, 1, 1.5, 2, 2.5, 3, 5, 6, 7, 10)],
    stat(index_mass_quantile, param=[{'q': q} for q in (.1, .2, .3, .4, .5, .6, .7, .8, .9)]),
    stat(symmetry_looking, param=[{'r': r*0.05} for r in range(20)])
)

In [12]:
np.sum([[1, 2, 3], [4, 5, 6]], axis=0)

array([5, 7, 9])

In [17]:
def add_quaternion_norm(X):
    X = X.copy()
    cols = ['orientation_X', 'orientation_Y', 'orientation_Z', 'orientation_W']
    X['quat_norm'] = np.sum([X[col]**2 for col in cols], axis=0)
    X['quat_mod'] = np.sqrt(X['quat_norm'])
    for col in cols:
        axis = col.split('_')[-1]
        X[f'norm_{axis}'] = X[col] / X['quat_mod']
    return X

In [18]:
def add_euler_angles(X):
    X = X.copy()
    x, y, z, w = [X[f'norm_{s}'] for s in list('XYZW')]
    nx, ny, nz = quaternion_to_euler(x, y, z, w)
    X['euler_X'] = nx
    X['euler_Y'] = ny
    X['euler_Z'] = nz
    return X

In [19]:
def quaternion_to_euler(x, y, z, w):
    t0 = 2.0*(w*x + y*z)
    t1 = 1.0 - 2.0*(x*x + y*y)
    X = np.arctan2(t0, t1)
    
    t2 = np.clip(2.0*(w*y - z*x), -1, 1)
    Y = np.arcsin(t2)
    
    t3 = 2.0*(w*z + x*y)
    t4 = 1.0 - 2.0*(y*y + z*z)
    Z = np.arctan2(t3, t4)
    
    return X, Y, Z

In [20]:
x_trn = add_euler_angles(add_quaternion_norm(x_trn)).drop(columns=[
    'orientation_X', 'orientation_Y', 'orientation_Z', 'orientation_W'])

In [21]:
x_tst = add_euler_angles(add_quaternion_norm(x_tst)).drop(columns=[
    'orientation_X', 'orientation_Y', 'orientation_Z', 'orientation_W'])

In [22]:
class StatsFeatures:
    def __init__(self, funcs=default_stats):
        self.funcs = funcs
    
    def __call__(self, data):
        features = {}
        for col in data.columns:
            for func in self.funcs:
                result = func(data[col].values) 
                if hasattr(result, '__len__'):
                    for key, value in result:
                        features[f'{col}__{func.__name__}__{key}'] = value
                else:
                    features[f'{col}__{func.__name__}'] = result
        features = {
            k: int(v) if v in (True, False) else v 
            for k, v in features.items()}
        return features

In [23]:
class SliceFeatures:
    def __init__(self, mode='first', n=5):
        if mode not in {'first', 'middle', 'last'}:
            raise ValueError('unexpected mode')
        self.mode = mode
        self.n = n
    
    def __call__(self, data):
        if self.mode == 'first':
            start, end = 0, self.n
        elif self.mode == 'last':
            start, end = -self.n, len(data)
        elif self.mode == 'middle':
            mid = len(data) // 2
            div, mod = divmod(self.n, 2)
            start, end = mid-div, mid+div+mod
        cols = data.columns
        vec = data.iloc[start:end].values.T.ravel()
        new_cols = [f'{col}_{self.mode}{i}' for i in range(self.n) for col in cols]
        return dict(zip(new_cols, vec))

In [24]:
_, group = next(iter(x_trn.groupby('series_id')))
group = group.drop(columns=['series_id', 'measurement_number'])

In [25]:
features = [
    StatsFeatures(),
    SliceFeatures('first'),
    SliceFeatures('middle'),
    SliceFeatures('last')
]

In [28]:
def generate_features(data, features, ignore=None):
    with Parallel(n_jobs=cpu_count()) as parallel:
        extracted = parallel(delayed(generate_features_for_group)(
            group=group.drop(columns=ignore or []),
            features=features
        ) for _, group in tqdm(data.groupby('series_id')))
    return pd.DataFrame(extracted)

In [29]:
def generate_features_for_group(group, features):
    return dict(ChainMap(*[feat(group) for feat in features]))

In [30]:
# generate_features_for_group(group, features)

In [31]:
ignore = ['series_id', 'measurement_number']

In [32]:
print('Feature extraction on train dataset')
x_trn_rich = generate_features(x_trn, features, ignore=ignore)

Feature extraction on train dataset


In [33]:
print('Feature extraction on train dataset')
x_tst_rich = generate_features(x_tst, features, ignore=ignore)

Feature extraction on train dataset


In [18]:
x_trn_rich.fillna(0, inplace=True)
x_trn_rich.replace(-np.inf, 0, inplace=True)
x_trn_rich.replace(+np.inf, 0, inplace=True)

In [19]:
x_tst_rich.fillna(0, inplace=True)
x_tst_rich.replace(-np.inf, 0, inplace=True)
x_tst_rich.replace(+np.inf, 0, inplace=True)

In [20]:
to_feather(x_trn_rich, 'trn_rich')

PosixPath('/home/ck/data/careercon2019/tmp/trn_rich.feather')

In [21]:
to_feather(x_tst_rich, 'tst_rich')

PosixPath('/home/ck/data/careercon2019/tmp/tst_rich.feather')

In [22]:
x_trn_rich, x_tst_rich, y_trn = from_feather('trn_rich', 'tst_rich', 'y_trn')

In [23]:
enc = LabelEncoder()
y = enc.fit_transform(y_trn['surface'])

In [24]:
relevance = calculate_relevance_table(x_trn_rich, pd.Series(y), ml_task='classification')

In [29]:
rel_cols = relevance[relevance['relevant']].index.tolist()

In [30]:
x_trn_rich = x_trn_rich[rel_cols]

In [31]:
x_tst_rich = x_tst_rich[rel_cols]

In [32]:
X_train, X_valid, y_train, y_valid = train_test_split(x_trn_rich, y, test_size=0.1)

In [33]:
def accuracy(y_true, y_pred):
    n = len(y_true)
    y_hat = y_pred.reshape(9, n).argmax(axis=0)
    value = (y_true == y_hat).mean()
    return 'accuracy', value, True

In [39]:
forest = RandomForestClassifier(
    n_estimators=1000, max_features='sqrt',
    max_depth=5, min_samples_leaf=5, min_samples_split=10,
    verbose=1, n_jobs=-1)

In [48]:
n_iter = 50

sampler = ParameterSampler({
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 1, 3, 5],
    'min_samples_split': [2, 4, 0.05],
    'min_samples_leaf': [1, 3, 0.05],
    'max_features': [0.6, 'sqrt', 'log2'],
}, n_iter=n_iter)

best_acc = 0
best_params = None

for i, params in enumerate(sampler):
    print(f'Sample {i+1:d}/{n_iter:d}')
    forest = RandomForestClassifier(n_estimators=100, random_state=seed, **params)
    forest.fit(X_train, y_train)
    y_hat = forest.predict(X_valid)
    acc = (y_hat == y_valid).mean()
    if acc > best_acc:
        print(f'\taccuracy improved: {acc:2.2%}')
        best_acc = acc
        best_params = params

Sample 1/50
	accuracy improved: 49.61%
Sample 2/50
	accuracy improved: 69.03%
Sample 3/50
Sample 4/50
Sample 5/50
	accuracy improved: 91.34%
Sample 6/50
Sample 7/50
Sample 8/50
Sample 9/50
Sample 10/50
Sample 11/50
Sample 12/50
Sample 13/50
Sample 14/50
Sample 15/50
Sample 16/50
Sample 17/50
Sample 18/50
Sample 19/50
Sample 20/50
Sample 21/50
Sample 22/50
Sample 23/50
Sample 24/50
Sample 25/50
Sample 26/50
Sample 27/50
Sample 28/50
Sample 29/50
Sample 30/50
Sample 31/50
Sample 32/50
Sample 33/50
Sample 34/50
Sample 35/50
Sample 36/50
Sample 37/50
Sample 38/50
Sample 39/50
Sample 40/50
Sample 41/50
Sample 42/50
Sample 43/50
Sample 44/50
Sample 45/50
Sample 46/50
Sample 47/50
Sample 48/50
Sample 49/50
Sample 50/50


In [49]:
best_params

{'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': None,
 'criterion': 'gini'}

In [34]:
model = lgb.LGBMClassifier(
    n_estimators=10000, learning_rate=0.1,
    colsample_bytree=0.4, objective='multiclass',
    num_leaves=500, num_class=9)

In [26]:
model.fit(
    X_train, y_train, 
    eval_set=[(X_valid, y_valid)], 
    eval_metric=accuracy,
    early_stopping_rounds=250,
    verbose=100)

Training until validation scores don't improve for 300 rounds.
[150]	valid_0's multi_logloss: 1.21663	valid_0's accuracy: 0.834646
[300]	valid_0's multi_logloss: 0.860089	valid_0's accuracy: 0.834646
[450]	valid_0's multi_logloss: 0.671321	valid_0's accuracy: 0.84252
[600]	valid_0's multi_logloss: 0.560852	valid_0's accuracy: 0.855643
[750]	valid_0's multi_logloss: 0.494286	valid_0's accuracy: 0.858268
[900]	valid_0's multi_logloss: 0.454289	valid_0's accuracy: 0.871391
[1050]	valid_0's multi_logloss: 0.427595	valid_0's accuracy: 0.87664
[1200]	valid_0's multi_logloss: 0.411023	valid_0's accuracy: 0.87664
Early stopping, best iteration is:
[1004]	valid_0's multi_logloss: 0.434309	valid_0's accuracy: 0.87664


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.4,
        importance_type='split', learning_rate=0.005, max_depth=-1,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=3000, n_jobs=-1, num_class=9, num_leaves=500,
        objective='multiclass', random_state=None, reg_alpha=0.0,
        reg_lambda=0.0, silent=True, subsample=1.0,
        subsample_for_bin=200000, subsample_freq=0)

In [27]:
imp = model.feature_importances_
idx = np.argsort(imp)[:100]

In [31]:
# f, ax = plt.subplots(1, 1, figsize=(8, 20))
# ax.barh(X_train.columns[idx], imp[idx])
# ax.set_title('Feature Importance');

In [32]:
submit = pd.read_csv(SAMPLE)
submit['surface'] = enc.inverse_transform(model.predict(x_tst_rich))
submit.to_csv('submit.csv', index=None)
!kaggle c submit career-con-2019 -f 'submit.csv' -m "LightGBM tsfresh + binary features"

100%|██████████████████████████████████████| 52.5k/52.5k [00:00<00:00, 46.9kB/s]
Successfully submitted to CareerCon 2019 - Help Navigate Robots 